In [ ]:
import sys
sys.path.append('/Users/liangwang/GitHub/DataAnalysis')

In [ ]:
import numpy as np
import sys, os
from pathlib import Path
import pandas as pd
import copy
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from plotly.colors import n_colors
import math
import datetime as dt
from sklearn.metrics import r2_score
import plotly.colors
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.interpolate import griddata
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
import scipy
import importlib
import pickle

# ----- Internal Dependencies -------#
nb_dir = os.path.split(os.getcwd())[0]
sys.path.append("../..")
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from General import FileRead
from General import AccuracyMetrics
from General.GeneralFunctions import get_add_to_dict
from General import TableManipulations
from FeatureGeneration import GeneralFeatures
from General import FileWrite

from Plot.PdfHelper import PdfHelper
from Plot.PlotMaker import PlotMaker
# from Plot import Voltage_Plots
from Plot import PlotFunctions
# from Plot import AdHocPlots
# from Plot import Sim_Plots

# widget and notebook stuff
from ipywidgets import interact, fixed
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

load data

In [ ]:
df=pd.read_csv('/Users/liangwang/Library/CloudStorage/OneDrive-Biolinq/Gen 1/Algorithm Development/Gen 1 Model Optimization/20240521/agg_mfg_ref_sample_df_from17.csv')
df_orig=df.copy()

Setting

In [259]:
# training_group = ['eBlinq17', 'eBlinq19a','eBlinq19b','eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqAcet','iBlinqFW231','iBlinqOverlay']
# test_group = ['Pre-Piv-2-training']
training_group = ['eBlinq19c','eBlinq20','eBlinq22','eBlinq23','eBlinq25','eBlinqRingOverlay&NiMNA','iBlinqFW231','iBlinqOverlay']
test_group = ['Pre-Piv-2-training']
training_time_window = [0.083,5]
all_training_channel = ['1','3','4']
# training_chemistry = '2.6.11'
# test_chemistry = '2.6.11'
iso_thresh = 25
feature_hour_window = '8hr'
feature_col = [
    'time_from_start_featureGeneration',
    f'ewm_average_{feature_hour_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'ewm_average_{feature_hour_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s1_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s3_glucoseFeatureGeneration',
    f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s4_glucoseFeatureGeneration',
]
gluc_col = [
    'model_output_s1_calculateGlucose','model_output_s3_calculateGlucose','model_output_s4_calculateGlucose',
]

gluc_ref_col = [
    'Study','Sensor',
    'retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose','cgm_adj','ysi','acck'
]

ground_truth_col = [
    'retro_slope_s1','retro_intercept_s1',
    'retro_slope_s3','retro_intercept_s3',
    'retro_slope_s4','retro_intercept_s4',
]
all_cols = feature_col + gluc_col + gluc_ref_col + ground_truth_col

In [ ]:
# df = df_orig[all_cols].copy()
# hypo_idx = df['cgm_adj'].le(100)
# for ch_str in all_training_channel:
#     df[f'ad_s{ch_str}'] = df[f'retro_model_output_s{ch_str}_calculateGlucose'].sub(df['cgm_adj']).abs()
#     df[f'ard_s{ch_str}'] = df[f'ad_s{ch_str}'].div(df['cgm_adj']).mul(100)
#     df[f'iso_s{ch_str}'] = df[f'ard_s{ch_str}']
#     df.loc[hypo_idx,f'iso_s{ch_str}'] = df.loc[hypo_idx,f'ad_s{ch_str}']
#     col_for_optim_no_ysi_acck = list(df.columns)
#     col_for_optim_no_ysi_acck.remove('ysi')
#     col_for_optim_no_ysi_acck.remove('acck')

# training_index = df[df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1])].index
# df_training = df.loc[training_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
# valid_training_index = df[df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & 
#                         df[f'iso_s1'].le(iso_thresh) & df[f'iso_s3'].le(iso_thresh) & df[f'iso_s4'].le(iso_thresh)].index
# df_training_valid = df.loc[valid_training_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)

# x_train = df_training_valid[feature_col]
# y_train = df_training_valid[ground_truth_col].to_numpy()
# min_max_scaler = sklearn.preprocessing.MinMaxScaler()
# x_train_norm = min_max_scaler.fit_transform(x_train)

In [260]:
df = df_orig[all_cols].copy()
col_for_optim_no_ysi_acck = list(df.columns)
col_for_optim_no_ysi_acck.remove('ysi')
col_for_optim_no_ysi_acck.remove('acck')

feature_name_avg_ch1 = f'ewm_average_{feature_hour_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_avg_ch1_lb = df[feature_name_avg_ch1].quantile(q=0.05)
feature_name_avg_ch1_ub = df[feature_name_avg_ch1].quantile(q=0.95)
feature_name_std_ch1 = f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s1_glucoseFeatureGeneration'
feature_name_std_ch1_lb = df[feature_name_std_ch1].quantile(q=0.05)
feature_name_std_ch1_ub = df[feature_name_std_ch1].quantile(q=0.95)
feature_name_avg_ch3 = f'ewm_average_{feature_hour_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_avg_ch3_lb = df[feature_name_avg_ch3].quantile(q=0.05)
feature_name_avg_ch3_ub = df[feature_name_avg_ch3].quantile(q=0.95)
feature_name_std_ch3 = f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s3_glucoseFeatureGeneration'
feature_name_std_ch3_lb = df[feature_name_std_ch3].quantile(q=0.05)
feature_name_std_ch3_ub = df[feature_name_std_ch3].quantile(q=0.95)
feature_name_avg_ch4 = f'ewm_average_{feature_hour_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_avg_ch4_lb = df[feature_name_avg_ch4].quantile(q=0.05)
feature_name_avg_ch4_ub = df[feature_name_avg_ch4].quantile(q=0.95)
feature_name_std_ch4 = f'estimate_standard_deviation_{feature_hour_window}_iir_filt_model_output_s4_glucoseFeatureGeneration'
feature_name_std_ch4_lb = df[feature_name_std_ch4].quantile(q=0.05)
feature_name_std_ch4_ub = df[feature_name_std_ch4].quantile(q=0.95)

training_index = df[df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1])].index
df_training = df.loc[training_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
valid_training_index = df[df['Study'].isin(training_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & 
                        df[feature_name_avg_ch1].between(feature_name_avg_ch1_lb,feature_name_avg_ch1_ub) & 
                        df[feature_name_std_ch1].between(feature_name_std_ch1_lb,feature_name_std_ch1_ub) &
                        df[feature_name_avg_ch3].between(feature_name_avg_ch3_lb,feature_name_avg_ch3_ub) &
                        df[feature_name_std_ch3].between(feature_name_std_ch3_lb,feature_name_std_ch3_ub) &
                        df[feature_name_avg_ch4].between(feature_name_avg_ch4_lb,feature_name_avg_ch4_ub) &
                        df[feature_name_std_ch4].between(feature_name_std_ch4_lb,feature_name_std_ch4_ub) &
                        df['retro_slope_s1'].between(0.5,1.5) & df['retro_intercept_s1'].between(-50,50) &
                        df['retro_slope_s3'].between(0.5,1.5) & df['retro_intercept_s3'].between(-50,50) &
                        df['retro_slope_s4'].between(0.5,1.5) & df['retro_intercept_s4'].between(-50,50)
                        ].index
df_training_valid = df.loc[valid_training_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
print(f'Training data has shape {df_training.shape}')
print(f'Training data has shape {df_training_valid.shape}')
good_data_ratio = len(df_training_valid)/len(df_training)
print(f'Ratio of training good data is {good_data_ratio}')
x_train = df_training_valid[feature_col]
y_train = df_training_valid[ground_truth_col].to_numpy()
min_max_scaler = sklearn.preprocessing.MinMaxScaler()
x_train_norm = min_max_scaler.fit_transform(x_train)

test_index = df[df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1])].index
df_test = df.loc[test_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
valid_test_index = df[df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & 
                        df[feature_name_avg_ch1].between(feature_name_avg_ch1_lb,feature_name_avg_ch1_ub) & 
                        df[feature_name_std_ch1].between(feature_name_std_ch1_lb,feature_name_std_ch1_ub) &
                        df[feature_name_avg_ch3].between(feature_name_avg_ch3_lb,feature_name_avg_ch3_ub) &
                        df[feature_name_std_ch3].between(feature_name_std_ch3_lb,feature_name_std_ch3_ub) &
                        df[feature_name_avg_ch4].between(feature_name_avg_ch4_lb,feature_name_avg_ch4_ub) &
                        df[feature_name_std_ch4].between(feature_name_std_ch4_lb,feature_name_std_ch4_ub) &
                        df['retro_slope_s1'].between(0.5,1.5) & df['retro_intercept_s1'].between(-50,50) &
                        df['retro_slope_s3'].between(0.5,1.5) & df['retro_intercept_s3'].between(-50,50) &
                        df['retro_slope_s4'].between(0.5,1.5) & df['retro_intercept_s4'].between(-50,50)
                        ].index
df_test_valid = df.loc[valid_test_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
print(f'Test data has shape {df_test.shape}')
print(f'Test data has shape {df_test_valid.shape}')
good_test_data_ratio = len(df_test_valid)/len(df_test)
print(f'Ratio of good test data is {good_test_data_ratio}')
x_test = df_test_valid[feature_col]
x_test_norm = min_max_scaler.fit_transform(x_test)
y_test = df_test_valid[ground_truth_col].to_numpy()

Training data has shape (4840579, 22)
Training data has shape (2339834, 22)
Ratio of training good data is 0.48337895115439705
Test data has shape (982014, 22)
Test data has shape (608644, 22)
Ratio of good test data is 0.6197915711995959


In [261]:
self_cal_model_dict = {}
for i in range(6): # [0,1]:
    print(f'Optimizing parameter {i}')
    self_cal_model = MLPRegressor(hidden_layer_sizes=(5,5),solver='sgd',alpha=1e-4,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528)
    self_cal_model.fit(x_train_norm,y_train[:,i])
    predict_slope_intercept = self_cal_model.predict(x_train_norm)
    mse_train_selfcal = np.mean((predict_slope_intercept - y_train[:,0])**2)
    mse_train_today = np.mean(y_train[:,i]**2)
    print(f'The model today has MSE={mse_train_today}')
    print(f'The self cal model has MSE={mse_train_selfcal}')

    predict_slope_intercept_test = self_cal_model.predict(x_test_norm)
    mse_test_selfcal = np.mean((predict_slope_intercept_test - y_test[:,i])**2)
    mse_test_today = np.mean(y_test[:,i]**2)
    print(f'The model today on test has MSE={mse_test_today}')
    print(f'The self cal model on test has MSE={mse_test_selfcal}')
    self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

Optimizing parameter 0
The model today has MSE=0.9193229441352048
The self cal model has MSE=0.03353375277271236
The model today on test has MSE=0.9434721182340603
The self cal model on test has MSE=0.02882043218824899
Optimizing parameter 1
The model today has MSE=404.09625527221516
The self cal model has MSE=38.3364787555681
The model today on test has MSE=384.8329433277382
The self cal model on test has MSE=386.0079922683138
Optimizing parameter 2
The model today has MSE=0.8700670865013446
The self cal model has MSE=0.0516444276449655
The model today on test has MSE=0.8788723484477182
The self cal model on test has MSE=0.02290377589146346
Optimizing parameter 3
The model today has MSE=393.0014112074821
The self cal model has MSE=42.556581042981826
The model today on test has MSE=353.29151324257407
The self cal model on test has MSE=336.4018987934296
Optimizing parameter 4
The model today has MSE=0.9185202128503553
The self cal model has MSE=0.05480728017626186
The model today on tes

In [262]:
ch1_slope_train = self_cal_model_dict['self_cal_model_0'].predict(x_train_norm)
ch1_intercept_train = self_cal_model_dict['self_cal_model_1'].predict(x_train_norm)
df_training_valid['model_output_s1_selfcal'] = df_training_valid['model_output_s1_calculateGlucose'].sub(ch1_intercept_train).div(ch1_slope_train)
mard_ch1_train_selfcal = df_training_valid['model_output_s1_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
mard_ch1_train_today = df_training_valid['model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
mard_ch1_train_retro = df_training_valid['retro_model_output_s1_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data MARD of ch1 glucose is {mard_ch1_train_today}')
print(f'Training data MARD of ch1 selfcal glucose is {mard_ch1_train_selfcal}')
print(f'Training data MARD of ch1 retro glucose is {mard_ch1_train_retro}')
ch1_slope_test = self_cal_model_dict['self_cal_model_0'].predict(x_test_norm)
ch1_intercept_test = self_cal_model_dict['self_cal_model_1'].predict(x_test_norm)
df_test_valid['model_output_s1_selfcal'] = df_test_valid['model_output_s1_calculateGlucose'].sub(ch1_intercept_test).div(ch1_slope_test)
mard_ch1_test_selfcal = df_test_valid['model_output_s1_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
mard_ch1_test_today = df_test_valid['model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
mard_ch1_test_retro = df_test_valid['retro_model_output_s1_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data MARD of ch1 glucose is {mard_ch1_test_today}')
print(f'Test data MARD of ch1 selfcal glucose is {mard_ch1_test_selfcal}')
print(f'Test data MARD of ch1 retro glucose is {mard_ch1_test_retro}')

Training data MARD of ch1 glucose is 0.21157109367696464
Training data MARD of ch1 selfcal glucose is 0.18547760017062262
Training data MARD of ch1 retro glucose is 0.16063806619825907
Test data MARD of ch1 glucose is 0.19695547916299005
Test data MARD of ch1 selfcal glucose is 0.17146476923280263
Test data MARD of ch1 retro glucose is 0.13405671217947096


In [295]:
min_max_scaler.data_min_

array([ 0.0830053, 46.8401   , 36.6114   , 42.1527   ,  5.79154  ,
        5.88723  ,  5.97644  ])

In [263]:
ch3_slope_train = self_cal_model_dict['self_cal_model_2'].predict(x_train_norm)
ch3_intercept_train = self_cal_model_dict['self_cal_model_3'].predict(x_train_norm)
df_training_valid['model_output_s3_selfcal'] = df_training_valid['model_output_s3_calculateGlucose'].sub(ch3_intercept_train).div(ch3_slope_train)
mard_ch3_train_selfcal = df_training_valid['model_output_s3_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
mard_ch3_train_today = df_training_valid['model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
mard_ch3_train_retro = df_training_valid['retro_model_output_s3_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data MARD of ch3 glucose is {mard_ch3_train_today}')
print(f'Training data MARD of ch3 selfcal glucose is {mard_ch3_train_selfcal}')
print(f'Training data MARD of ch3 retro glucose is {mard_ch3_train_retro}')
ch3_slope_test = self_cal_model_dict['self_cal_model_2'].predict(x_test_norm)
ch3_intercept_test = self_cal_model_dict['self_cal_model_3'].predict(x_test_norm)
df_test_valid['model_output_s3_selfcal'] = df_test_valid['model_output_s3_calculateGlucose'].sub(ch3_intercept_test).div(ch3_slope_test)
mard_ch3_test_selfcal = df_test_valid['model_output_s3_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
mard_ch3_test_today = df_test_valid['model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
mard_ch3_test_retro = df_test_valid['retro_model_output_s3_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data MARD of ch3 glucose is {mard_ch3_test_today}')
print(f'Test data MARD of ch3 selfcal glucose is {mard_ch3_test_selfcal}')
print(f'Test data MARD of ch3 retro glucose is {mard_ch3_test_retro}')

Training data MARD of ch3 glucose is 0.2225366095403253
Training data MARD of ch3 selfcal glucose is 0.1894561146148285
Training data MARD of ch3 retro glucose is 0.16345391185336255
Test data MARD of ch3 glucose is 0.19537829743436028
Test data MARD of ch3 selfcal glucose is 0.17138015184454383
Test data MARD of ch3 retro glucose is 0.137533313099084


In [264]:
ch4_slope_train = self_cal_model_dict['self_cal_model_4'].predict(x_train_norm)
ch4_intercept_train = self_cal_model_dict['self_cal_model_5'].predict(x_train_norm)
df_training_valid['model_output_s4_selfcal'] = df_training_valid['model_output_s4_calculateGlucose'].sub(ch4_intercept_train).div(ch4_slope_train)
mard_ch4_train_selfcal = df_training_valid['model_output_s4_selfcal'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
mard_ch4_train_today = df_training_valid['model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
mard_ch4_train_retro = df_training_valid['retro_model_output_s4_calculateGlucose'].sub(df_training_valid['cgm_adj']).div(df_training_valid['cgm_adj']).abs().mean()
print(f'Training data MARD of ch4 glucose is {mard_ch4_train_today}')
print(f'Training data MARD of ch4 selfcal glucose is {mard_ch4_train_selfcal}')
print(f'Training data MARD of ch4 retro glucose is {mard_ch4_train_retro}')
ch4_slope_test = self_cal_model_dict['self_cal_model_4'].predict(x_test_norm)
ch4_intercept_test = self_cal_model_dict['self_cal_model_5'].predict(x_test_norm)
df_test_valid['model_output_s4_selfcal'] = df_test_valid['model_output_s4_calculateGlucose'].sub(ch4_intercept_test).div(ch4_slope_test)
mard_ch4_test_selfcal = df_test_valid['model_output_s4_selfcal'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
mard_ch4_test_today = df_test_valid['model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
mard_ch4_test_retro = df_test_valid['retro_model_output_s4_calculateGlucose'].sub(df_test_valid['cgm_adj']).div(df_test_valid['cgm_adj']).abs().mean()
print(f'Test data MARD of ch4 glucose is {mard_ch4_test_today}')
print(f'Test data MARD of ch4 selfcal glucose is {mard_ch4_test_selfcal}')
print(f'Test data MARD of ch4 retro glucose is {mard_ch4_test_retro}')

Training data MARD of ch4 glucose is 0.2113779375457251
Training data MARD of ch4 selfcal glucose is 0.18534839065679082
Training data MARD of ch4 retro glucose is 0.16258662897338938
Test data MARD of ch4 glucose is 0.19349628958680995
Test data MARD of ch4 selfcal glucose is 0.17145969506036488
Test data MARD of ch4 retro glucose is 0.13784931008953544


In [273]:
df_test_valid[['model_output_s1_selfcal','model_output_s3_selfcal','model_output_s4_selfcal']].std(1).quantile(q=0.95)

25.29394231821411

In [274]:
df_test_valid[['model_output_s1_calculateGlucose','model_output_s3_calculateGlucose','model_output_s4_calculateGlucose']].std(1).quantile(q=0.95)

47.869971580377985

In [275]:
df_test_valid[['retro_model_output_s1_calculateGlucose','retro_model_output_s3_calculateGlucose','retro_model_output_s4_calculateGlucose']].std(1).quantile(q=0.95)

28.335525535464043

In [ ]:
self_cal_model_temp = MLPRegressor(solver='sgd',alpha=1e-1,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528,verbose=True)
param_list = {"hidden_layer_sizes": [(10,),(10,10),(10,5,2),(2,5,10),(4,3,4),(3,2,3),(5,5),(5,)]}
gridCV = GridSearchCV(estimator=self_cal_model_temp, param_grid=param_list,cv=10,verbose=True)
gridCV.fit(x_train_norm,y_train[:,i])

In [ ]:
pd.DataFrame(gridCV.cv_results_)

In [ ]:
gridCV.best_estimator_

In [ ]:
i=1
self_cal_model = MLPRegressor(hidden_layer_sizes=(1,),solver='sgd',alpha=0.1,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528)
self_cal_model.fit(x_train_norm,y_train[:,i])
predict_slope_intercept = self_cal_model.predict(x_train_norm)
mse_train_selfcal = np.mean((predict_slope_intercept - y_train[:,0])**2)
mse_train_today = np.mean(y_train[:,i]**2)
print(f'The model today has MSE={mse_train_today}')
print(f'The self cal model has MSE={mse_train_selfcal}')

predict_slope_intercept_test = self_cal_model.predict(x_test_norm)
mse_test_selfcal = np.mean((predict_slope_intercept_test - y_test[:,i])**2)
mse_test_today = np.mean(y_test[:,i]**2)
print(f'The model today on test has MSE={mse_test_today}')
print(f'The self cal model on test has MSE={mse_test_selfcal}')
self_cal_model_dict[f'self_cal_model_{i}'] = self_cal_model

In [ ]:
y_train[:,i].mean()

In [ ]:
df_training_valid[feature_col].plot(y='ewm_average_8hr_iir_filt_model_output_s3_glucoseFeatureGeneration',kind='hist',bins=1000)

In [ ]:
min_max_scaler.data_max_

In [ ]:
self_cal_model = MLPRegressor(hidden_layer_sizes=(10,10,10),solver='sgd',alpha=1e-4,max_iter=int(1e4),tol=1e-6,learning_rate='adaptive',random_state=20240528)
self_cal_model.fit(x_train_norm,y_train[:,0])
predict_slope_intercept = self_cal_model.predict(x_train_norm)

In [ ]:
valid_test_index = df[df['Study'].isin(test_group) & df['time_from_start_featureGeneration'].between(training_time_window[0],training_time_window[1]) & 
                        df[f'iso_s{ch_str}'].le(iso_thresh) & df[f'iso_s3'].le(iso_thresh) & df[f'iso_s4'].le(iso_thresh)].index
df_test_valid = df.loc[valid_test_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
print(f'Test data has shape {df_test_valid.shape}')
x_test = df_test_valid[feature_col]
x_test_norm = min_max_scaler.fit_transform(x_test)
y_test = df_test_valid[ground_truth_col].to_numpy()

predict_slope_intercept_test = self_cal_model.predict(x_test_norm)

In [ ]:
y_test[:,0].max()

In [ ]:
np.mean((y_test[:,0])**2)
np.mean((predict_slope_intercept_test - y_test[:,0])**2)

In [ ]:
self_cal_model.loss_

In [ ]:
y_train[:,0]

In [ ]:
predict_slope_intercept

In [ ]:
self_cal_model.predict(x_train_norm[10000,].reshape(1,-1))

In [ ]:
predict_slope_intercept.shape

In [ ]:

    print(f'Training data has shape {df_training.shape}')
    print(f'Training data has shape {df_training_valid.shape}')
    good_data_ratio = len(df_training_valid)/len(df_training)
    print(f'Ratio of good data is {good_data_ratio}')
    valid_test_index = df[df['Study'].isin(test_group) & df['time_from_start'].between(training_time_window[0],training_time_window[1]) & 
                            df[f'iso_s{ch_str}'].le(iso_thresh) & df['Chemistry'].eq(training_chemistry)].index
    df_test_valid = df.loc[valid_test_index,col_for_optim_no_ysi_acck].dropna().reset_index(drop=True)
    print(f'Test data has shape {df_test_valid.shape}')
    g_train = df_training_valid['cgm_adj']
    features_train = df_training_valid[['time_from_start',f'mag{ch_str}_1007']]
    cur_train = df_training_valid[f'cur{ch_str}']

    g_test = df_test_valid['cgm_adj']
    features_test = df_test_valid[['time_from_start',f'mag{ch_str}_1007']]
    cur_test = df_test_valid[f'cur{ch_str}']

    result = scipy.optimize.dual_annealing(mard,params_bound,callback=callback_da,args=(g_train,cur_train,features_train,ch_str),maxiter=int(1e3))

    this_result.append(mard(result.x,g_train,cur_train,features_train,ch_str))
    this_result.append(mard(result.x,g_test,cur_test,features_test,ch_str))

    params_dict = params_list_to_dict(result.x,ch_str)
    g_hat_train, _ = glucose_model(cur_train,features_train,params_dict) 
    g_hat_test, _ = glucose_model(cur_test,features_test,params_dict) 

    reg_train = LinearRegression().fit(g_train.values.reshape(-1,1), g_hat_train.values)
    this_result.append(reg_train.score(g_train.values.reshape(-1,1), g_hat_train.values))
    this_result.append(reg_train.coef_)
    this_result.append(reg_train.intercept_)           

    reg_test = LinearRegression().fit(g_test.values.reshape(-1,1), g_hat_test.values)
    this_result.append(reg_test.score(g_test.values.reshape(-1,1), g_hat_test.values))
    this_result.append(reg_test.coef_)
    this_result.append(reg_test.intercept_)

    all_optim_params[f'channel_{ch_str}'] = params_dict
    all_results[f'channel_{ch_str}'] = this_result
    
results_df = pd.DataFrame.from_dict(all_results)
results_df.to_csv(r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\optimize_time_varying_model.csv',header=True)
with open (r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\all_optim_params.p','wb') as fp:
    pickle.dump(all_optim_params,fp)    

In [ ]:
with open (r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\all_optim_params.p','wb') as fp:
    pickle.dump(all_optim_params,fp)

In [ ]:
# with open (r'C:\Users\lwang\OneDrive - Biolinq Inc\Gen 1\Algorithm Development\Gen 1 Model Optimization\20240521\all_optim_params.p','rb') as fp:
#     all_optim_params2 = pickle.load(fp)

In [ ]:
all_results

In [ ]:
all_optim_params

In [ ]:
import scipy.stats as dist
dist_info_sim_slope = dist.gamma(a=16,loc=0,scale=1/7061)
time_from_start = [0,0.08,0.25,0.5,1,2,5]
sim_slope_sensitivity = 0 + 1*dist_info_sim_slope.cdf(time_from_start)
print(sim_slope_sensitivity) 

In [ ]:
result

In [ ]:
import pprint
pp = pprint.PrettyPrinter(depth=4)
params_dict = params_list_to_dict(result.x,1)
pp.pprint(params_dict)

In [ ]:
params = params_list_to_dict(result.x,ch_str)
g_train_hat, sensitivity = glucose_model(cur_train,features_train,params)    
mard = g_train_hat.sub(g_train).abs().div(g_train).mean()
error = g_train_hat.sub(g_train).mul(sensitivity).abs().div(g_train).mean()

In [ ]:
mard

In [ ]:
error

In [ ]:
sim_slope = g_train.div(cur_train).div(features_train['mag1_1007']).mean()
sim_slope

In [ ]:
g_simple = cur_train.mul(features_train['mag1_1007']).mul(sim_slope)
g_simple.sub(g_train).abs().div(g_train).mean()

In [ ]:
reg_simple = LinearRegression().fit(g_train.values.reshape(-1,1), g_simple.values)
print(reg_simple.score(g_train.values.reshape(-1,1), g_simple.values))
print(reg_simple.coef_)
print(reg_simple.intercept_)

In [ ]:
params = params_list_to_dict(result.x,ch_str)
g_hat_train, _ = glucose_model(cur_train,features_train,params) 
g_hat_test, _ = glucose_model(cur_test,features_test,params) 
reg_train = LinearRegression().fit(g_train.values.reshape(-1,1), g_hat_train.values)
print(reg_train.score(g_train.values.reshape(-1,1), g_hat_train.values))
print(reg_train.coef_)
print(reg_train.intercept_)